## Simple tensor with random items

In [2]:
import numpy as np

# Scientific notation
np.set_printoptions(suppress=True)

params = np.random.uniform(low=-50, high=150, size=20)

params[0] = params.max() + 1
params[1] = params.min() - 1
params[2] = 0

params = np.round(params, 2)

print(params)

[142.52 -44.56   0.   -28.02 -28.41 124.52  30.04  17.55 113.82 -43.56
 123.82 111.96  70.22 141.52  82.62 -37.24  52.53  15.24 111.85 -13.85]


## Quantization methods and quantize

In [8]:
def clamp(params_q: np.array, lower_bound: int, upper_bound: int) -> np.array:
    params_q[params_q < lower_bound] = lower_bound
    params_q[params_q > upper_bound] = upper_bound
    return params_q

def asymmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float, int]:
    # scale and zero point
    alpha = np.max(params)
    beta = np.min(params)
    scale = (alpha - beta) / (2**bits - 1)
    zero = -1 * np.round(beta / scale)
    lower_bound, upper_bound = 0, 2**bits - 1
    # Quantize the paramaters
    quantized = clamp(np.round(params / scale + zero), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero

def asymmetric_dequantize(params_q: np.array, scale: float, zero: int) -> np.array:
    return (params_q - zero) * scale

def symmetric_dequantize(params_q: np.array, scale: float) -> np.array:
    return params_q * scale

def symmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float]:
    # Calculate the scale
    alpha = np.max(np.abs(params))
    scale = alpha / (2**(bits-1)-1)
    lower_bound = -2**(bits-1)
    upper_bound = 2**(bits-1)-1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale

def quantization_error(params: np.array, params_q: np.array):
    # Calculate the MSE
    return np.mean((params - params_q) ** 2)

(asymetric_q, asymetric_scale, asymetric_zero) = asymmetric_quantization(params, 8)
(symetric_q, symetric_scale) = symmetric_quantization(params, 8)

print("Original:")
print(np.round(params, 2))
print('')
print(f"Asymetric scale; {asymetric_scale}, zero: {asymetric_zero}")
print(asymetric_q)
print('')
print(f"Symetric_scale: {symetric_scale}")
print(symetric_q)

Original:
[142.52 -44.56   0.   -28.02 -28.41 124.52  30.04  17.55 113.82 -43.56
 123.82 111.96  70.22 141.52  82.62 -37.24  52.53  15.24 111.85 -13.85]

Asymetric scale; 0.7336470588235294, zero: 61.0
[255   0  61  23  22 231 102  85 216   2 230 214 157 254 174  10 133  82
 213  42]

Symetric_scale: 1.1222047244094489
[127 -40   0 -25 -25 111  27  16 101 -39 110 100  63 126  74 -33  47  14
 100 -12]


In [9]:
# Dequantize the parameters back to 32 bits
params_deq_asymetric = asymmetric_dequantize(asymetric_q, asymetric_scale, asymetric_zero)
params_deq_symetric = symmetric_dequantize(symetric_q, symetric_scale)

print("Original:")
print(np.round(params, 2))
print('')
print("Dequantize Asymetric:")
print(np.round(params_deq_asymetric))
print('')
print("Dequantize Symetric:")
print(np.round(params_deq_symetric, 2))

Original:
[142.52 -44.56   0.   -28.02 -28.41 124.52  30.04  17.55 113.82 -43.56
 123.82 111.96  70.22 141.52  82.62 -37.24  52.53  15.24 111.85 -13.85]

Dequantize Asymetric:
[142. -45.   0. -28. -29. 125.  30.  18. 114. -43. 124. 112.  70. 142.
  83. -37.  53.  15. 112. -14.]

Dequantize Symetric:
[142.52 -44.89   0.   -28.06 -28.06 124.56  30.3   17.96 113.34 -43.77
 123.44 112.22  70.7  141.4   83.04 -37.03  52.74  15.71 112.22 -13.47]


In [10]:
# Quantization error
print(f'{"Asymmetric error: ":>20}{np.round(quantization_error(params, params_deq_asymetric), 2)}')
print(f'{"Symmetric error: ":>20}{np.round(quantization_error(params, params_deq_symetric), 2)}')

  Asymmetric error: 0.04
   Symmetric error: 0.1
